In [1]:
import os
import sys
from pathlib import Path

import numpy as np
import pandas as pd 

import matplotlib.pyplot as plt
import seaborn as sns

import wot
import scanpy as sc
import scvelo as scv
import cellrank as cr
import anndata as ad
from cellrank.external.kernels import WOTKernel
from cellrank.estimators import GPCCA
#from tueplots import bundles
#from tueplots import axes


In [2]:
cr.logging.print_versions()

cellrank==1.5.1+g525b847 scanpy==1.9.1 anndata==0.8.0 numpy==1.23.5 numba==0.56.4 scipy==1.10.0 pandas==1.5.2 pygpcca==1.0.4 scikit-learn==1.2.0 statsmodels==0.13.5 scvelo==0.2.5 pygam==0.8.0 matplotlib==3.6.2 seaborn==0.12.2


In [22]:
adata825 = sc.read("/lustre/groups/ml01/workspace/monge_velo/data/benchmarks/gastrulation/stage_825_growth_rate.h5ad")
adata85 = sc.read("/lustre/groups/ml01/workspace/monge_velo/data/benchmarks/gastrulation/stage_85_growth_rate.h5ad")
adata = ad.concat([adata825, adata85], merge = 'same')

leave_in_cell = ['Blood progenitors 2', 'Erythroid1', 'Erythroid2', 'Erythroid3']
adata = adata[adata.obs["celltype"].isin(leave_in_cell)].copy()

sc.pp.log1p(adata)

adata.obs = adata.obs.loc[:, ['stage', 'celltype']]
adata.var = adata.var.loc[:, ['highly_variable']]
adata.obs['celltype'] = pd.Series(adata.obs['celltype'], dtype='category')

In [ ]:
wk = WOTKernel(adata, time_key="stage")
wk.compute_initial_growth_rates(organism="mouse", key_added="growth_rate_init")
wk.compute_transition_matrix(
    growth_iters=3, growth_rate_key="growth_rate_init"
)

  0%|                                                                                      | 0/1 [00:00<?, ?time pair/s]

In [ ]:
g = GPCCA(wk)

In [ ]:
g.compute_schur(n_components=20)
g.plot_spectrum(real_only=True)

In [ ]:
g.compute_macrostates(n_states=1, cluster_key="celltype")
scv.set_figure_params('scvelo', transparent=True, fontsize=20, color_map='viridis')
g.plot_macrostates(discrete=True, basis="umap", legend_loc="right", legend_fontweight='normal', legend_fontsize='12', dpi=250)

In [21]:
g.set_terminal_states_from_macrostates(
    [
        'Erythroid3'
    ]
)
g.compute_absorption_probabilities(solver="gmres", use_petsc=True, tol=1e-12, preconditioner='ilu', time_to_absorption='all')
g.plot_absorption_probabilities(same_plot=False, basis="umap", perc=[0, 99], ncols=2)

TypeError: compute_absorption_probabilities() got an unexpected keyword argument 'time_to_absorption'

In [ ]:
cr.pl.aggregate_absorption_probabilities(
    adata,
    mode='heatmap',
    lineages=['Erythroid3'],
    cluster_key='celltype',
    clusters=['Blood progenitors 2', 'Erythroid1', 'Erythroid2', 'Erythroid3'],
    figsize=(17, 7),
    title="",
    #save="transprobs_wot.png"
)